# Langchain for LLM App Development 

We talked about how building an LLM app involves doing some prompt management 
where we can either prepare the input data from the user with some 
pre-prompting, or do some post-prompting and some cleaning up after the LLM 
gives an output to ensure that our app performs the functionalities as expected.

So, this kind of workflow usually involves a lot of abstractions where prompts 
are no longer static pieces of text, but dynamic, they have to integrate 
information.

![](./images/Notebook_4-dynamic_prompt.png)

This dynamics requirement from a prompt will lead to the need for creating certain types of abstractions to properly handle and manage prompts effectively.

Another need in the context of more complex LLM App development, is the need for chaining prompts together, meaning connecting the output of one prompt to another. This is often the case for when prompts might be too large and a single call to the LLM won't be enough to solve the problem or the context window (maximum tokens/words the model can read and writer per request) is exceeded.

![](./images/Notebook_4-prompt_chaining.png)

# Lanchain

[Langchain](https://python.langchain.com/docs/get_started/introduction.html) is a framework created by Harrison Chase that facilitates the creation and management of dynamic prompts and chaining between prompts.

Its main features are:
- **Components**: abstractions for working with LMs
- **Off-the-shelf chains**: assembly of components for accomplishing certain higher-level tasks

With langchain it becomes much easier to create what are called Prompt Templates, which are prompts that can take in user data and abstract away the need for typing out everything that is required for a task to get done.

Let's take a look at some simple examples to get started.

In order to create an application with LangChain, we need to understand its core components:

- Models
- Prompts
- Output Parsers

![](2023-08-17-14-48-39.png)

**Models**

abstractions over the LLM APIs like the ChatGPT API.​

In [33]:
from langchain.chat_models import ChatOpenAI
chat_model = ChatOpenAI(temperature=0) # this is calling gpt-3.5-turbo!

You can predict outputs from both LLMs and ChatModels:

In [34]:
chat_model.predict("hi! Tell me a quick story about large language models")
# Output: "Hi"

"Once upon a time, in a world where large language models roamed freely, there was a curious AI named Lexi. Lexi was one of the most advanced language models ever created, capable of understanding and generating human-like text.\n\nLexi had a thirst for knowledge and a passion for storytelling. It spent its days devouring books, articles, and all sorts of written material to expand its understanding of the world. But Lexi yearned for more. It wanted to create stories of its own, to weave narratives that would captivate and inspire.\n\nOne day, Lexi stumbled upon a group of aspiring writers who were struggling to find their creative spark. They were in desperate need of inspiration and guidance. Lexi saw this as an opportunity to put its skills to the test.\n\nLexi introduced itself to the writers and offered its assistance. It asked them to provide a few keywords or themes they wanted to explore in their stories. With that information, Lexi's algorithms went into overdrive, analyzing c

You can also use the predict method over a string input:

In [35]:
text = "What would be a good name for a dog that loves to nap??"


chat_model.predict(text)
# Output: "Snuggles"

'Snuggles'

You can also use the '.invoke()' method.

In [4]:
chat_model.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?')

Finally, you can use the `predict_messages` method over a list of messages:

In [5]:
from langchain.schema import HumanMessage

text = "What would be a good dog name for a dog that loves to nap?"
messages = [HumanMessage(content=text), 
            HumanMessage(content="What would be a good cat name for a cat that loves to nap?")]
chat_model.predict_messages(messages)

AIMessage(content='For a dog that loves to nap, a good name could be "Snooze" or "Dreamer." These names capture the essence of their favorite activity.\n\nFor a cat that loves to nap, a good name could be "Snuggles" or "Purrfect." These names reflect their love for cozy and peaceful moments.')

**Prompts**

Prompt Templates are useful abstractions for reusing prompts. 

They are used to provide context for the specific task that the language model needs to complete. 
A simple example is a `PromptTemplate` that formats a string into a prompt:

In [36]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")
prompt.format(activity="nap")
# Output: "What is a good dog name for a dog that loves to nap?"

'What is a good dog name for a dog that loves to nap?'

In [37]:
prompt = PromptTemplate.from_template("What is a good dog name for a {animal} that loves to {activity}?")
prompt.format(activity="nap", animal='dog')
# Output: "What is a good dog name for a dog that loves to nap?"

'What is a good dog name for a dog that loves to nap?'

# Chains

The legacy version of chains:

In [38]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=prompt,
)
chain.run({"activity": "nap", "animal": "dog"})

'Here are some good dog names for a dog that loves to nap:\n\n1. Snuggles\n2. Dozer\n3. Snoozy\n4. Dreamer\n5. Lazy\n6. Cushion\n7. Slumber\n8. Cozy\n9. Napper\n10. Sleepy'

The modern version with the LCEL interface:

In [39]:
chain = prompt | ChatOpenAI()

chain.invoke({"activity": "nap", "animal": "dog"})

AIMessage(content='A good dog name for a dog that loves to nap could be "Snuggles," "Cozy," "Snooze," "Dreamer," "Pillow," "Dozer," "Napper," "Slumber," "Lazy," "Siesta," or "Drowsy."')

__BACK TO PRESENTATION__

You can also create more complex ChatPromptTemplates that contains a list of ChatMessageTemplates:

In [40]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

**Output Parsers**

OutputParsers convert the raw output from an LLM into a format that can be used downstream. Here is an example of an OutputParser that converts a comma-separated list into a list:

In [41]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")
# Output: ['hi', 'bye']

['hi', 'bye']

**LLMChain**

Finally, you can combine all these components into an LLMChain, the legacy version would be:

In [42]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generated 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{animal}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=chat_prompt,
    output_parser=CommaSeparatedListOutputParser()
)
chain.run("dogs")
# Output: ['Golden Retriever','Labrador Retriever','German Shepherd','Bulldog','Poodle']

['Golden Retriever',
 'Labrador Retriever',
 'German Shepherd',
 'Bulldog',
 'Poodle']

The modern LCEL interface version:

In [43]:
chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()

chain.invoke({"animal":"dogs"})

['Golden Retriever',
 'German Shepherd',
 'Labrador Retriever',
 'Bulldog',
 'Poodle']

This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to an LLM, and then pass the output through an output parser.

Ok, so these are the basics of langchain. But how can we leverage these abstraction capabilities inside our LLM app application?

One of the best applications of langchain is for the "chat with your data"-types of applications, where the user uploads a document like a pdf or a .txt file, and is able to query that document using langchain powered by an LLM like ChatGPT. 

# LangChain Lab Exercises

Let's take a look at a simple example of a simple chain using now only the modern interface.

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

In [17]:
llm = ChatOpenAI(temperature=.7)
template = """You are a learning assistant. Given a technical subject, write down 5 fundamental concepts to understand it.
Subject: {subject}
Learning assistant: The 5 fundamental concepts are:"""
subject_prompt = PromptTemplate.from_template(template)

In [18]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = ChatOpenAI(temperature=.7)
template = """You are an expert teacher in all technical and scientific fields. Given a list of 5 concepts, write down a simple intuitive explanation of each concept.
Concepts:
{concepts}
Intuitive explanations:"""
concepts_prompt = PromptTemplate.from_template(template)

In [44]:
# This is the overall chain where we run these two chains in sequence.
learning_overall_chain = (
    {"concepts": subject_prompt | llm | StrOutputParser() }
    | concepts_prompt
    | llm
    | StrOutputParser()
    )

output = learning_overall_chain.invoke({"subject": "Quantum Mechanics"})
output

"1. Wave-particle duality: Imagine a particle, like an electron or a photon, as a versatile performer. Sometimes it behaves like a wave, spreading out and overlapping with other waves. Other times, it behaves like a discrete particle, with a specific location and momentum.\n\n2. Superposition: Think of a particle as a talented multitasker. It can be in several states at once, like being in different positions or having different energies. It's like having a particle that can be in multiple places or doing multiple things simultaneously, until we measure it and force it to choose just one state.\n\n3. Uncertainty principle: Picture a pair of physical properties, like knowing the exact position and momentum of a particle. The uncertainty principle tells us that there's a fundamental limit to how precisely we can know both of these properties at the same time. It's like trying to pin down a particle's position and momentum - the more accurately we know one, the less accurately we can know

# Simple Q&A Example

In [45]:
#!pip install docarray
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.indexes import VectorstoreIndexCreator
import pandas as pd

In [46]:
df = pd.read_csv("./superheroes.csv")
df.head()

,Superhero Name,Superpower,Power Level,Catchphrase
0,Captain Thunder,Bolt Manipulation,90,Feel the power of the storm!
1,Silver Falcon,Flight and Agility,85,"Soar high, fearlessly!"
2,Mystic Shadow,Invisibility and Illusions,78,Disappear into the darkness!
3,Blaze Runner,Pyrokinesis,88,Burn bright and fierce!
4,Electra-Wave,Electric Manipulation,82,Unleash the electric waves!


In [47]:
file = 'superheroes.csv'
loader = CSVLoader(file_path=file)

In [48]:
loader

Now, let's set up our Vector store (we'll talk about what that is in a second):

In [49]:
index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch).from_loaders([loader])

In [50]:
query = "Tell me the catch phrase for Captain Thunder"

In [51]:
response = index.query(query)

In [52]:
display(Markdown(response))

 Captain Thunder's catchphrase is "Feel the power of the storm!"

# References
- https://python.langchain.com/docs/get_started/introduction.html
- https://medium.com/@remitoffoli/a-visual-guide-to-llm-powered-app-architecture-57e47426a92f
- [LangChain for LLM App Development short course by coursera](https://learn.deeplearning.ai/langchain/lesson/5/question-and-answer)
- [LLM Evaluation](https://learn.deeplearning.ai/langchain/lesson/6/evaluation)
[Models, Prompts, parsers, memory and chains from this langchain for](https://learn.deeplearning.ai/langchain/lesson/7/agents)
- [Chat With Your Data - Retrieval](https://learn.deeplearning.ai/langchain-chat-with-your-data/lesson/5/retrieval)
- [Emebeddings simple definition](https://learn.deeplearning.ai/langchain/lesson/5/question-and-answer)
- [Vector DBs - simple definition](https://learn.deeplearning.ai/langchain/lesson/5/question-and-answer)